# Neural Language Modeling

## 1. Data Processing
## 2. Encoder: Bag of Words
## 3. Decoder: MLP
## 4. Training Neural LM
## 5. Sentence Score
## 6. Sentence Generation

In [2]:
# !wget http://ikulikov.name/neural_lm_amazon_model_N10.pt
# !wget http://ikulikov.name/neural_lm_amazon_model_N7.pt
# !wget http://ikulikov.name/neural_lm_amazon_model_N5.pt
# !wget http://ikulikov.name/neural_lm_amazon_model_N3.pt

In [9]:
# !pip install altair
# !pip install pygtrie
# !pip install spacy
!pip install nvidia-ml-py3

  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-cp37-none-any.whl size=19192 sha256=6e463541038d71424e76a03e9e68957eaf41e42211657abb4842fc2ceb033ac3
  Stored in directory: /home/jupyter/.cache/pip/wheels/e4/1d/06/640c93f5270d67d0247f30be91f232700d19023f9e66d735c7
Successfully built nvidia-ml-py3


In [10]:
!ls

amazon_dataset.ipynb  neural_lm.ipynb	     ngram_lm_small.ipynb
c_utils		      neural_lm_small.ipynb
ken_lm.ipynb	      ngram_lm.ipynb


In [11]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

     |████████████████████████████████| 12.0MB 779kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-cp37-none-any.whl size=12019125 sha256=7b86749a1f61ea817a31586c592b02f5fcfe2cecb534ea340975c6ad09e1d567
  Stored in directory: /home/jupyter/.cache/pip/wheels/48/5c/1c/15f9d02afc8221a668d2172446dd8467b20cdb9aef80a172a4
Successfully built en-core-web-sm


In [7]:
# !pip install -U spacy
# !spacy download en_core_web_sm
# !pip install torchtext

ERROR: Could not find a version that satisfies the requirement parlai (from versions: none)
ERROR: No matching distribution found for parlai


Uncomment the code bellow to install parlAI

In [3]:
# git clone https://github.com/facebookresearch/ParlAI.git ./ParlAI
# cd ParlAI/
# python setup.py develop

In [4]:
import os
import sys
sys.path.append('c_utils/')
import torch
import numpy as np
import torch.nn as nn
import random
import math

import c_utils.ngram_utils as ngram_utils
from c_utils.ngram_utils import NgramLM
from c_utils.amazon_dataset import AmazonDataset, pad, batchify
from torch.utils.data import DataLoader
from c_utils.neural_lm import BagOfNGrams, DecoderMLP, seq2seq
import c_utils.global_variables as gl
import torch
from tqdm import tqdm_notebook, tqdm
_tqdm = tqdm_notebook

In [6]:
torch.manual_seed(1)

In [7]:
USE_CUDA = True
device = torch.device("cuda" if (torch.cuda.is_available() and USE_CUDA) else "cpu")

# Data Processing

In [8]:
# Read data from .txt files and create lists of reviews
train_data = []
# create a list of all the reviews 
with open('../../data/train.txt', 'r') as f:
    train_data = [review for review in f.read().split('\n') if review]
    
valid_data = []
# create a list of all the reviews 
with open('../../data/valid.txt', 'r') as f:
    valid_data = [review for review in f.read().split('\n') if review]
    

In [9]:
train_data[0], valid_data[0]
train_data = train_data#[:100]
valid_data = valid_data#[:10]
train_data[0], type(train_data), len(train_data), type(train_data[0])

("this is a great tutu and at a really great price . it doesn ' t look cheap at all . i ' m so glad i looked on amazon and found such an affordable tutu that isn ' t made poorly . a + + ",
 list,
 222919,
 str)

In [10]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)


In [37]:
N = 3 # 5, 7, 10

In [38]:
train_data_padded = ngram_utils.pad_dataset(train_data_tokenized, n=N)
valid_data_padded = ngram_utils.pad_dataset(valid_data_tokenized, n=N)

In [39]:
vocab = ngram_utils.get_vocab(train_data_padded)
vocab_size = len(vocab)
vocab_size, vocab[:10]

(63993, ('<sos>', '<eos>', '.', 'the', 'i', ',', 'and', 'a', 'it', 'to'))

In [40]:
id2token, token2id = ngram_utils.get_dict(vocab)
len(id2token), len(token2id)

(63996, 63994)

In [41]:
train_data_ids = ngram_utils.get_ids(train_data_padded, token2id)
valid_data_ids = ngram_utils.get_ids(valid_data_padded, token2id)

In [42]:
BATCH_SIZE = 2048

In [ ]:
train_dataset = AmazonDataset(train_data_ids, max_inp_length=None, use_cuda=USE_CUDA)
train_dataset_ngrams = []
for t in train_dataset:
    for i in range(len(t) - N):
        train_dataset_ngrams.append((t[i:i + N], t[i + N]))
train_loader = DataLoader(train_dataset_ngrams, batch_size=BATCH_SIZE, collate_fn=batchify, shuffle=True)

 98%|█████████▊| 1044278/1069724 [03:33<00:00, 29112.60it/s]

In [ ]:
valid_dataset = AmazonDataset(valid_data_ids, max_inp_length=None, use_cuda=USE_CUDA)
valid_dataset_ngrams = []
for t in valid_dataset:
    for i in range(len(t) - N):
        valid_dataset_ngrams.append((t[i:i + N], t[i + N]))
valid_loader = DataLoader(valid_dataset_ngrams, batch_size=BATCH_SIZE, collate_fn=batchify, shuffle=True)

In [ ]:
num_train = len(train_dataset_ngrams)
num_valid = len(valid_dataset_ngrams)
num_train, num_valid

In [ ]:
EMB_DIM = 300
HIDDEN_SIZE = 256
ENC_SIZE = 128
ACTIVATION = 'ReLU'
NUM_LAYERS = 2
REDUCE = 'mean'
DROPOUT = 0.1
LEARNING_RATE = 0.1
BATCH_NORM = False

# Encoder

In [ ]:
encoder = BagOfNGrams(len(id2token), emb_dim=EMB_DIM, hidden_size=HIDDEN_SIZE, out_size=ENC_SIZE, activation=ACTIVATION, \
                        nlayers=NUM_LAYERS, reduce=REDUCE, dropout=DROPOUT, batch_norm=BATCH_NORM)
encoder

# Decoder

In [ ]:
decoder = DecoderMLP(input_size=ENC_SIZE, output_size=len(id2token), hidden_size=HIDDEN_SIZE)
decoder

# Model

In [ ]:
model = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
model

In [ ]:
TRAIN = True

# Training

In [ ]:
if TRAIN:
    num_epochs = 10
    log_interval = 1000
    best_eval_loss = np.inf

    for epoch in range(num_epochs):
        # Train on entire train data using minibatches
        cur_loss = 0
        for i, (data, labels) in enumerate(train_loader):
            prediction, loss = model.train_step(data, labels)
            cur_loss += loss

            if i % log_interval == 0 and i > 0:
                cur_loss = cur_loss / log_interval
                print('| Epoch {:3d} | Train Loss {:5.2f} | Train PPL {:8.2f} | {:5d}/{:5d} Batches'.format(
                    epoch, cur_loss, math.exp(cur_loss), i, int(num_train/len(data))))
                cur_loss = 0

        # Eval at the end of every epoch
        eval_loss = 0
        for i, (data, labels) in enumerate(valid_loader):
            prediction, loss = model.eval_step(data, labels)
            eval_loss += len(data) * loss
        eval_loss = eval_loss / num_valid 
        print('-' * 89)
        print('| Epoch {:3d} | Valid Loss {:5.2f} | Valid PPL {:8.2f}'.format(
            epoch, eval_loss, math.exp(eval_loss)))
        print('-' * 89)

        # Save the model if the validation loss is the best we've seen so far.
        if not best_eval_loss or eval_loss < best_eval_loss:
            model.save_model('neural_lm_amazon_model_all_new_N{}'.format(N) + '.pt')
            best_eval_loss = eval_loss
            
    pretrained_model = model

else:
    # Load Pretrained Model
    pretrained_model = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
    pretrained_model.load_model('neural_lm_amazon_model_all_new_N{}'.format(N) + '.pt')
    pretrained_model

### Get Loader for Some N

In [ ]:
def get_loader(train_data_tokenized, valid_data_tokenized, N):
    # Tokenize the Datasets
    # TODO: this takes a really long time !! why?
    train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
    valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)
    
    train_data_padded = ngram_utils.pad_dataset(train_data_tokenized, n=N)
    valid_data_padded = ngram_utils.pad_dataset(valid_data_tokenized, n=N)
    
    vocab = ngram_utils.get_vocab(train_data_padded)
    id2token, token2id = ngram_utils.get_dict(vocab)
    
    train_data_ids = ngram_utils.get_ids(train_data_padded, token2id)
    valid_data_ids = ngram_utils.get_ids(valid_data_padded, token2id)

    train_dataset = AmazonDataset(train_data_ids, max_inp_length=None, use_cuda=USE_CUDA)
    train_dataset_ngrams = []
    for t in train_dataset:
        for i in range(len(t) - N):
            train_dataset_ngrams.append((t[i:i + N], t[i + N]))
    train_loader = DataLoader(train_dataset_ngrams, batch_size=BATCH_SIZE, collate_fn=batchify, shuffle=True)

    valid_dataset = AmazonDataset(valid_data_ids, max_inp_length=None, use_cuda=USE_CUDA)
    valid_dataset_ngrams = []
    for t in valid_dataset:
        for i in range(len(t) - N):
            valid_dataset_ngrams.append((t[i:i + N], t[i + N]))
    valid_loader = DataLoader(valid_dataset_ngrams, batch_size=BATCH_SIZE, collate_fn=batchify, shuffle=True)
    
    num_train = len(train_dataset_ngrams)
    num_valid = len(valid_dataset_ngrams)
    
    return train_loader, num_train, valid_loader, num_valid

## Perplexity (Train + Valid Data)

In [ ]:
def get_perplexity(loader, num_data, model):
    total_loss = 0
    total_count = 0
    for i, (data, labels) in enumerate(loader):
        prediction, loss = model.eval_step(data, labels)
        total_loss += len(data) * loss
        total_count += len(data)
    total_loss = total_loss / total_count
    ppl = math.exp(total_loss)
    return ppl

In [ ]:
# Load Pretrained Model
N = 3
print("Ngrams: ", N)

model_n3 = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
model_n3.load_model('neural_lm_amazon_model_all_N{}'.format(N) + '.pt')
print("Model: ", model_n3)

train_loader, num_train, valid_loader, num_valid = get_loader(train_data_tokenized, valid_data_tokenized, N)

valid_ppl = get_perplexity(valid_loader, num_valid, model_n3)
print("Validation Perplexity: ", valid_ppl)

train_ppl = get_perplexity(train_loader, num_train, model_n3)
print("Training Perplexity: ", train_ppl)


In [36]:
# Load Pretrained Model
N = 5
print("Ngrams: ", N)

model_n5 = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
model_n5.load_model('neural_lm_amazon_model_all_N{}'.format(N) + '.pt')
print("Model: ", model_n5)

train_loader, num_train, valid_loader, num_valid = get_loader(train_data_tokenized, valid_data_tokenized, N)

valid_ppl = get_perplexity(valid_loader, num_valid, model_n5)
print("Validation Perplexity: ", valid_ppl)

train_ppl = get_perplexity(train_loader, num_train, model_n5)
print("Training Perplexity: ", train_ppl)


Ngrams:  5


FileNotFoundError: [Errno 2] No such file or directory: 'neural_lm_amazon_model_all_N5.pt'

In [ ]:
# # Load Pretrained Model
# N = 7
# print("Ngrams: ", N)

# model_n7 = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
# model_n7.load_model('neural_lm_amazon_model_all_N{}'.format(N) + '.pt')
# print("Model: ", model_n7)

# train_loader, num_train, valid_loader, num_valid = get_loader(train_data_tokenized, valid_data_tokenized, N)

# valid_ppl = get_perplexity(valid_loader, num_valid, model_n7)
# print("Validation Perplexity: ", valid_ppl)

# train_ppl = get_perplexity(train_loader, num_train, model_n7)
# print("Training Perplexity: ", train_ppl)


In [ ]:
# # Load Pretrained Model
# N = 10
# print("Ngrams: ", N)

# model_n10 = seq2seq(encoder, decoder, id2token, use_cuda=USE_CUDA, lr=LEARNING_RATE, size_ngrams=N) 
# model_n10.load_model('neural_lm_amazon_model_all_N{}'.format(N) + '.pt')
# print("Model: ", model_n10)

# train_loader, num_train, valid_loader, num_valid = get_loader(train_data_tokenized, valid_data_tokenized, N)

# valid_ppl = get_perplexity(valid_loader, num_valid, model_n10)
# print("Validation Perplexity: ", valid_ppl)

# train_ppl = get_perplexity(train_loader, num_train, model_n10)
# print("Training Perplexity: ", train_ppl)


## Score Sentences

In [ ]:
def score_sentence(sent, model):
    tokenized, _ = ngram_utils.tokenize_dataset(sent)
    sent_ids = ngram_utils.get_ids(tokenized, token2id)
    sent_tensor = torch.LongTensor(sent_ids).to(device)
    generated, scores = model.evaluate(sent_tensor, score_only=True)
    ppl = math.exp(scores)
    return ppl

In [ ]:
sentence = ['i like pandas']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
# ppl7 = score_sentence(sentence, model_n7)
# ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5#, ppl7, ppl10

In [ ]:
sentence = ['tutu tutu is not my favorit']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
# ppl7 = score_sentence(sentence, model_n7)
# ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5#, ppl7, ppl10

In [ ]:
sentence = ['i really like this watch']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
# ppl7 = score_sentence(sentence, model_n7)
# ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5#, ppl7, ppl10

In [ ]:
sentence = ['training neural networks']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
# ppl7 = score_sentence(sentence, model_n7)
# ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5#, ppl7, ppl10

In [ ]:
sentence = ['this is a great tutu']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
# ppl7 = score_sentence(sentence, model_n7)
# ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5#, ppl7, ppl10

In [ ]:
sentence = ['my wife really likes the color of this dress']
ppl3 = score_sentence(sentence, model_n3)
ppl5 = score_sentence(sentence, model_n5)
# ppl7 = score_sentence(sentence, model_n7)
# ppl10 = score_sentence(sentence, model_n10)
ppl3, ppl5#, ppl7, ppl10

## Generate Sentences

In [ ]:
def generate_sentence(model, context=None):
    if context is None:
        dummy_context = torch.LongTensor([[0]]).to(device)
        generated, scores = model.evaluate(dummy_context, use_context=False)
    else:
        tokenized, _ = ngram_utils.tokenize_dataset(context)
        context_ids = ngram_utils.get_ids(tokenized, token2id)
        context_tensor = torch.LongTensor(context_ids).to(device)
        generated, scores = model.evaluate(context_tensor, use_context=True)
    
    ppl = math.exp(scores)
    return generated, scores

### No Context

In [ ]:
generated, scores = generate_sentence(model_n3)
print(' '.join(word[0] for word in generated)), scores

In [ ]:
generated, scores = generate_sentence(model_n5)
print(' '.join(word[0] for word in generated))

In [ ]:
# generated, scores = generate_sentence(model_n7)
# print(' '.join(word[0] for word in generated))

In [ ]:
# generated, scores = generate_sentence(model_n10)
# print(' '.join(word[0] for word in generated))

### With Context

In [ ]:
generated, scores = generate_sentence(model_n3, context=['this dress'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n5, context=['this dress'])
print(' '.join(word[0] for word in generated))

In [ ]:
# generated, scores = generate_sentence(model_n7, context=['this dress'])
# print(' '.join(word[0] for word in generated))

In [ ]:
# generated, scores = generate_sentence(model_n10, context=['this dress'])
# print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n3, context=['i like'])
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(model_n5, context=['i like'])
print(' '.join(word[0] for word in generated))

In [ ]:
# generated, scores = generate_sentence(model_n7, context=['i like'])
# print(' '.join(word[0] for word in generated))

In [ ]:
# generated, scores = generate_sentence(model_n10, context=['i like'])
# print(' '.join(word[0] for word in generated))